In [1]:
from __future__ import print_function, division
import os
import torch
import pandas as pd
from skimage import io, transform
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
import pickle

# Ignore warnings
import warnings
warnings.filterwarnings("ignore")

plt.ion()   # interactive mode

In [2]:
def onehot(vals, possible_vals):
    if not isinstance(possible_vals, list): raise TypeError("provide possible_vals as a list")
    enc_vals = np.zeros([len(vals), len(possible_vals)])
    for i, value in enumerate(vals):
        if isinstance(possible_vals[0], float):
            enc = np.where(abs(possible_vals-value)<1e-3)
        else:
            enc = np.where(possible_vals==value)
        enc_vals[i,enc] = 1
    return enc_vals

In [3]:
class Rescale(object):
    def __init__(self, scalar):
        self.scalar = scalar

    def __call__(self, im):
        w, h = [int(s*self.scalar) for s in im.size]
        return transforms.Resize((h, w))(im)

class Crop(object):
    def __init__(self, box):
        assert len(box) == 4
        self.box = box

    def __call__(self, im):
        return im.crop(self.box)

class Augment(object):
    def __init__(self, seq):
        self.seq = seq

    def __call__(self, im):
        return Image.fromarray(self.seq.augment_images([np.array(im)])[0])

def get_data_transforms(t='train'):
    data_transforms = {
        'train': transforms.Compose([Crop((0,120,800,480)),
            Rescale(0.4),
            transforms.ToTensor()
        ]),
        'val': transforms.Compose([
            Crop((0,120,800,480)),
            Rescale(0.4),
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ]),
    }
    return data_transforms[t]

In [23]:
from torch.utils.data import Dataset
from matplotlib import image
from matplotlib import pyplot
from torchvision import transforms
from PIL import Image

#LABEL_KEYS = ['v_throttle', 'v_break', 'v_steer', 'traffic_light', 'front_vehicle', 'centre_dist', 'pedestrian_distance']

class CARLA_Dataset(Dataset):
    def __init__(self, t, pickle_file_path_l, image_dir_path_l, pickle_file_path_r, image_dir_path_r):
        #initializing transforms
        assert t in ['train', 'val']
        self.transform = get_data_transforms(t)
        
        ###############LEFT CAMERA##############
        #reading the title of all images to access the pickle file
        self.image_path_l = image_dir_path_l
        self.image_list_l = os.listdir(image_dir_path_l)
        self.file_name_l = []
        for file in self.image_list_l:
            self.file_name_l.append(os.path.splitext(file)[0])
            
        #initialize the labels which should be returned
        self.labels_l = {}
        
        #read pickle file
        self.pickle_list_l = os.listdir(pickle_file_path_l)
        self.pickled_data_l = {}
        for file in self.pickle_list_l:
            f = open((pickle_file_path_l + file), 'rb')  
            self.pickled_data_l.update(pickle.load(f))
            f.close() 
        
        ###############RIGHT CAMERA##############
        #reading the title of all images to access the pickle file
        self.image_path_r = image_dir_path_r
        self.image_list_r = os.listdir(image_dir_path_r)
        self.file_name_r = []
        for file in self.image_list_r:
            #self.file_name.append(file[0:16])
            self.file_name_r.append(os.path.splitext(file)[0])
            
        #initialize the labels which should be returned
        self.labels_r = {}
        
        #read pickle file
        self.pickle_list_r = os.listdir(pickle_file_path_r)
        self.pickled_data_r = {}
        for file in self.pickle_list_r:
            f = open((pickle_file_path_r + file), 'rb')  
            self.pickled_data_r.update(pickle.load(f))
            f.close() 
        
        # transform reg affordances
        #reg_keys = ['front_vehicle', 'centre_dist', 'pedestrian_distance']
        #for key in self.pickled_data:
        #   entry = self.pickled_data[key]
            #print("1.", entry['front_vehicle'])
        #reg_norm = entry[reg_keys] / abs(entry[reg_keys]).max()
                # transform cls affordances
      
        f.close() 

    def __len__(self):
        return len(self.file_name_l)

    def __getitem__(self, idx):

        #####################LEFT CAMERA################
        #reading PIL
        self.raw_image_l = Image.open(os.path.join(self.image_path_l, self.image_list_l[idx]))
        
        #reading file name to access the pickle file
        current_fname_l = self.file_name_l[idx]
        label_dict_l = self.pickled_data_l[current_fname_l]       
        label_values_l = list(label_dict_l.values())
        #print(label_values)
        print(label_dict_l)        
        
        
        #transforming classification labels
        self.labels_l['traffic_light'] = torch.Tensor(onehot(np.array(label_dict_l['traffic_light']), [False, True]))
        
        #transforming raw PIL image
        im_l = self.transform(self.raw_image_l)
        
        ###############RIGHT CAMERA####################
        #reading PIL
        self.raw_image_r = Image.open(os.path.join(self.image_path_r, self.image_list_r[idx]))
        
        #reading file name to access the pickle file
        current_fname_r = self.file_name_r[idx]
        label_dict_r = self.pickled_data_r[current_fname_r]       
        label_values_r = list(label_dict_r.values())
        #print(label_values)
        print(label_dict_r)        
        
        
        #transforming classification labels
        self.labels_r['traffic_light'] = torch.Tensor(onehot(np.array(label_dict_r['traffic_light']), [False, True]))
        
        #transforming raw PIL image
        im_r = self.transform(self.raw_image_r)
        
        
        #return transformed image and labels
        sample = {'image_l' : im_l, 'labels_l' : label_values_l,'image_r' : im_r, 'labels_r' : label_values_r}
        return sample
        
        #self.labels['v_break'] = torch.Tensor(onehot(np.array(list(label_dict['v_break'])), [0.0, 1.0]))
        #print(label_dict['traffic_light'])
        #print(self.labels['traffic_light'])
        #print(label_dict['v_break'])
        #print(self.labels['v_break'])
        #pyplot.imshow(self.raw_image)
        #pyplot.show()
        #reg_keys = ('front_vehicle', 'centre_dist', 'pedestrian_distance', 'v_steer')
        #reg_norm = label_dict[reg_keys] / abs(label_dict[reg_keys]).max()
        #reg_norm = label_dict[reg_keys]

        #for i in 'front_vehicle', 'centre_dist', 'pedestrian_distance', 'v_steer':
        #  reg_norm = label_dict[i] / abs(label_dict[i])
        #  print(i, reg_norm)
        #print(type(reg_keys))
        #reg_norm = label_dict['front_vehicle', 'centre_dist'] / max(abs(label_dict['front_vehicle', 'centre_dist']))

In [24]:
C_dataset = CARLA_Dataset( 'train', pickle_file_path_l='/home/seima/CE7454_2019/codes/data/Left_pickle_file/',
                                    image_dir_path_l='/home/seima/CE7454_2019/codes/data/Left_camera/', 
                                    pickle_file_path_r='/home/seima/CE7454_2019/codes/data/Right_pickle_file/',
                                    image_dir_path_r='/home/seima/CE7454_2019/codes/data/Right_camera/', )

for i in range(len(C_dataset)):
  sample = C_dataset[i]
  print(sample['image_l'])
  print(sample['labels_l'])

{'333259.163962255': {'traffic_light': (False, 'Green'), 'centre_dist': 0.44513560798296037, 'v_steer': 0.00022016253205947578, 'front_vehicle': 22.957376844753714, 'v_throttle': 1.0, 'v_break': 0.0, 'pedestrian_distance': 50}, '334396.902331668': {'traffic_light': (False, 'Green'), 'centre_dist': 0.07424669896816907, 'v_steer': 0.004347591195255518, 'front_vehicle': 35.11325120565561, 'v_throttle': 0.5, 'v_break': 0.0, 'pedestrian_distance': 50}, '333827.507799709': {'traffic_light': (False, 'Green'), 'centre_dist': 0.15299610068557248, 'v_steer': -1.678466833254788e-05, 'front_vehicle': 22.577636744475566, 'v_throttle': 0.5, 'v_break': 0.0, 'pedestrian_distance': 50}, '335845.228966817': {'traffic_light': (False, 'Green'), 'centre_dist': 0.14096579052877573, 'v_steer': -3.967285010730848e-05, 'front_vehicle': 12.158085128592042, 'v_throttle': 0.5, 'v_break': 0.0, 'pedestrian_distance': 50}, '333691.84147426': {'traffic_light': (False, 'Green'), 'centre_dist': 0.15825225171544205, 'v_

{'v_throttle': 0.5, 'v_break': 0.0, 'front_vehicle': 50, 'pedestrian_distance': 50, 'traffic_light': (False, 'Green'), 'v_steer': 0.0006152561982162297, 'centre_dist': 0.4398108371613459}
tensor([[[0.0549, 0.0549, 0.0588,  ..., 0.0667, 0.0510, 0.0627],
         [0.0196, 0.0275, 0.0314,  ..., 0.0588, 0.0549, 0.0627],
         [0.0196, 0.0235, 0.0235,  ..., 0.0588, 0.0549, 0.0588],
         ...,
         [0.4118, 0.4196, 0.4235,  ..., 0.4824, 0.3725, 0.3569],
         [0.4196, 0.4196, 0.4235,  ..., 0.4863, 0.3804, 0.3608],
         [0.4118, 0.4118, 0.4196,  ..., 0.4863, 0.3843, 0.3725]],

        [[0.0980, 0.1020, 0.1059,  ..., 0.0706, 0.0549, 0.0627],
         [0.0510, 0.0627, 0.0745,  ..., 0.0627, 0.0588, 0.0627],
         [0.0471, 0.0471, 0.0588,  ..., 0.0627, 0.0588, 0.0588],
         ...,
         [0.3843, 0.3882, 0.3961,  ..., 0.4471, 0.3412, 0.3255],
         [0.3882, 0.3922, 0.3961,  ..., 0.4471, 0.3490, 0.3333],
         [0.3843, 0.3882, 0.3922,  ..., 0.4471, 0.3529, 0.3412]],



{'v_throttle': 1.0, 'v_break': 0.0, 'front_vehicle': 50, 'pedestrian_distance': 50, 'traffic_light': (False, 'Green'), 'v_steer': 1.3950892935099546e-05, 'centre_dist': 0.3541492278007678}
tensor([[[0.5686, 0.5686, 0.5647,  ..., 0.3020, 0.3020, 0.3020],
         [0.5725, 0.5725, 0.5725,  ..., 0.3059, 0.3098, 0.3059],
         [0.5804, 0.5804, 0.5765,  ..., 0.3098, 0.3059, 0.3059],
         ...,
         [0.4118, 0.4196, 0.4275,  ..., 0.3608, 0.3529, 0.3412],
         [0.4235, 0.4275, 0.4235,  ..., 0.3294, 0.3333, 0.3412],
         [0.4392, 0.4392, 0.4353,  ..., 0.3137, 0.3098, 0.3176]],

        [[0.6314, 0.6314, 0.6314,  ..., 0.2980, 0.2980, 0.2980],
         [0.6353, 0.6353, 0.6353,  ..., 0.2980, 0.3020, 0.3020],
         [0.6392, 0.6392, 0.6392,  ..., 0.3020, 0.2980, 0.2980],
         ...,
         [0.3843, 0.3922, 0.4000,  ..., 0.3333, 0.3255, 0.3137],
         [0.3961, 0.4000, 0.3961,  ..., 0.3020, 0.3059, 0.3137],
         [0.4078, 0.4078, 0.4039,  ..., 0.2863, 0.2824, 0.2863]],


{'v_throttle': 1.0, 'v_break': 0.0, 'front_vehicle': 50, 'pedestrian_distance': 50, 'traffic_light': (False, 'Green'), 'v_steer': -0.0001966203999472782, 'centre_dist': 0.44461134081237064}
tensor([[[0.0314, 0.0235, 0.0353,  ..., 0.1176, 0.1176, 0.1216],
         [0.0392, 0.0353, 0.0431,  ..., 0.1176, 0.1176, 0.1176],
         [0.0314, 0.0353, 0.0471,  ..., 0.1176, 0.1137, 0.1137],
         ...,
         [0.4353, 0.4353, 0.4314,  ..., 0.3725, 0.3765, 0.3647],
         [0.4353, 0.4392, 0.4353,  ..., 0.3490, 0.3529, 0.3529],
         [0.4392, 0.4392, 0.4353,  ..., 0.3216, 0.3412, 0.3412]],

        [[0.0549, 0.0471, 0.0627,  ..., 0.1176, 0.1216, 0.1216],
         [0.0667, 0.0627, 0.0667,  ..., 0.1176, 0.1176, 0.1176],
         [0.0549, 0.0588, 0.0667,  ..., 0.1176, 0.1137, 0.1137],
         ...,
         [0.4078, 0.4078, 0.4039,  ..., 0.3451, 0.3451, 0.3373],
         [0.4078, 0.4078, 0.4039,  ..., 0.3216, 0.3255, 0.3255],
         [0.4078, 0.4118, 0.4078,  ..., 0.2941, 0.3098, 0.3098]],

tensor([[[0.5529, 0.5529, 0.5529,  ..., 0.2353, 0.2353, 0.2353],
         [0.5569, 0.5569, 0.5569,  ..., 0.2353, 0.2353, 0.2353],
         [0.5333, 0.5216, 0.5098,  ..., 0.2431, 0.2392, 0.2353],
         ...,
         [0.4078, 0.4118, 0.4196,  ..., 0.3608, 0.3647, 0.3647],
         [0.4118, 0.4196, 0.4196,  ..., 0.3490, 0.3412, 0.3451],
         [0.4157, 0.4196, 0.4157,  ..., 0.3176, 0.3294, 0.3255]],

        [[0.6235, 0.6235, 0.6235,  ..., 0.2314, 0.2353, 0.2353],
         [0.6275, 0.6275, 0.6275,  ..., 0.2314, 0.2314, 0.2353],
         [0.6000, 0.5843, 0.5765,  ..., 0.2392, 0.2353, 0.2314],
         ...,
         [0.3804, 0.3843, 0.3961,  ..., 0.3333, 0.3333, 0.3373],
         [0.3843, 0.3922, 0.3922,  ..., 0.3216, 0.3137, 0.3176],
         [0.3882, 0.3922, 0.3882,  ..., 0.2902, 0.3020, 0.2980]],

        [[0.7255, 0.7255, 0.7255,  ..., 0.2980, 0.3020, 0.2980],
         [0.7255, 0.7294, 0.7294,  ..., 0.3020, 0.2980, 0.2980],
         [0.6824, 0.6588, 0.6431,  ..., 0.3059, 0.3020, 0.

...

tensor([[[0.5686, 0.5647, 0.5647,  ..., 0.6510, 0.6275, 0.5725],
         [0.5725, 0.5725, 0.5686,  ..., 0.5020, 0.3176, 0.1765],
         [0.5804, 0.5804, 0.5765,  ..., 0.0980, 0.0196, 0.0118],
         ...,
         [0.4353, 0.4471, 0.4549,  ..., 0.3608, 0.3569, 0.3608],
         [0.4471, 0.4471, 0.4510,  ..., 0.3569, 0.3569, 0.3647],
         [0.4471, 0.4471, 0.4510,  ..., 0.3569, 0.3608, 0.3647]],

        [[0.6314, 0.6314, 0.6314,  ..., 0.5686, 0.5490, 0.4902],
         [0.6353, 0.6353, 0.6353,  ..., 0.4235, 0.2588, 0.1333],
         [0.6392, 0.6392, 0.6392,  ..., 0.0706, 0.0118, 0.0078],
         ...,
         [0.4039, 0.4157, 0.4235,  ..., 0.3294, 0.3294, 0.3333],
         [0.4157, 0.4157, 0.4196,  ..., 0.3294, 0.3294, 0.3373],
         [0.4157, 0.4157, 0.4196,  ..., 0.3294, 0.3294, 0.3333]],

        [[0.7255, 0.7294, 0.7294,  ..., 0.3686, 0.3490, 0.3020],
         [0.7255, 0.7255, 0.7294,  ..., 0.2588, 0.1451, 0.0667],
         [0.7294, 0.7294, 0.7294,  ..., 0.0353, 0.0078, 0.

{'v_throttle': 1.0, 'v_break': 0.0, 'front_vehicle': 50, 'pedestrian_distance': 50, 'traffic_light': (False, 'Green'), 'v_steer': -0.00015607562090735883, 'centre_dist': 0.44658751257611584}
tensor([[[0.5647, 0.5647, 0.5647,  ..., 0.0039, 0.0196, 0.0549],
         [0.5725, 0.5686, 0.5686,  ..., 0.0118, 0.0235, 0.0549],
         [0.5765, 0.5765, 0.5765,  ..., 0.0157, 0.0275, 0.0627],
         ...,
         [0.0980, 0.1020, 0.0980,  ..., 0.1882, 0.1686, 0.1608],
         [0.0980, 0.0941, 0.0941,  ..., 0.2863, 0.2745, 0.2667],
         [0.0980, 0.0941, 0.0941,  ..., 0.3373, 0.3176, 0.3059]],

        [[0.6314, 0.6314, 0.6314,  ..., 0.0157, 0.0392, 0.0784],
         [0.6353, 0.6353, 0.6353,  ..., 0.0196, 0.0392, 0.0784],
         [0.6392, 0.6392, 0.6392,  ..., 0.0275, 0.0431, 0.0824],
         ...,
         [0.1216, 0.1255, 0.1216,  ..., 0.1804, 0.1608, 0.1569],
         [0.1216, 0.1176, 0.1137,  ..., 0.2627, 0.2510, 0.2431],
         [0.1216, 0.1137, 0.1176,  ..., 0.3098, 0.2902, 0.2784]]

{'v_throttle': 0.5, 'v_break': 0.0, 'front_vehicle': 50, 'pedestrian_distance': 50, 'traffic_light': (False, 'Green'), 'v_steer': 0.005009896587580442, 'centre_dist': 0.36315083762317346}
tensor([[[0.2745, 0.2627, 0.2784,  ..., 0.0392, 0.0510, 0.0588],
         [0.3020, 0.2941, 0.3176,  ..., 0.0588, 0.0588, 0.0627],
         [0.3333, 0.3294, 0.3333,  ..., 0.0588, 0.0549, 0.0627],
         ...,
         [0.4902, 0.4824, 0.4784,  ..., 0.3529, 0.3529, 0.3569],
         [0.4706, 0.4667, 0.4667,  ..., 0.3608, 0.3529, 0.3490],
         [0.4588, 0.4549, 0.4627,  ..., 0.3686, 0.3647, 0.3608]],

        [[0.3020, 0.2863, 0.3020,  ..., 0.0392, 0.0549, 0.0667],
         [0.3294, 0.3255, 0.3451,  ..., 0.0627, 0.0627, 0.0667],
         [0.3647, 0.3647, 0.3686,  ..., 0.0627, 0.0588, 0.0667],
         ...,
         [0.4588, 0.4549, 0.4471,  ..., 0.3255, 0.3216, 0.3255],
         [0.4392, 0.4353, 0.4353,  ..., 0.3294, 0.3255, 0.3216],
         [0.4235, 0.4235, 0.4314,  ..., 0.3412, 0.3373, 0.3333]],



{'v_throttle': 1.0, 'v_break': 0.0, 'front_vehicle': 50, 'pedestrian_distance': 50, 'traffic_light': (False, 'Green'), 'v_steer': -0.00013482230133377016, 'centre_dist': 0.4398615534540889}
tensor([[[0.5647, 0.5647, 0.5647,  ..., 0.3608, 0.3569, 0.3569],
         [0.5725, 0.5686, 0.5686,  ..., 0.3608, 0.3569, 0.3569],
         [0.5765, 0.5765, 0.5765,  ..., 0.3608, 0.3569, 0.3569],
         ...,
         [0.4235, 0.4196, 0.4235,  ..., 0.3490, 0.3490, 0.3412],
         [0.4392, 0.4392, 0.4392,  ..., 0.3451, 0.3529, 0.3490],
         [0.4275, 0.4275, 0.4275,  ..., 0.3608, 0.3569, 0.3529]],

        [[0.6314, 0.6314, 0.6314,  ..., 0.3412, 0.3373, 0.3373],
         [0.6353, 0.6353, 0.6353,  ..., 0.3412, 0.3373, 0.3373],
         [0.6392, 0.6392, 0.6392,  ..., 0.3412, 0.3373, 0.3373],
         ...,
         [0.3961, 0.3922, 0.3961,  ..., 0.3216, 0.3216, 0.3098],
         [0.4078, 0.4078, 0.4118,  ..., 0.3176, 0.3216, 0.3176],
         [0.4000, 0.4000, 0.4000,  ..., 0.3294, 0.3294, 0.3255]],

IndexError: list index out of range

In [ ]:
print(len(C_dataset))